<a href="https://colab.research.google.com/github/scarscarin/creativecoding/blob/main/%F0%9F%96%95_how_to_give_google_the_middle_finger_%F0%9F%96%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to give Google the middle finger 🖕

If you want to give Google the middle finger, you have many options:
- you can go the closest Google HQ and flip the receptionist off
- you can open [google.com](https://google.com) and show your finger 🖕
- you can train a model on a self-made dataset of middle fingers, using Google Colab, Google Mediapipe, and Google Drive!

👉 We are going for the third, assuming you already made a middle fingers dataset.

<hr>


## Step 0 (zero). Upload your dataset to Google Drive
Upload your middle finger dataset to your [Google Drive](https://drive.google.com).

Make a folder ``gestures_dataset`` with the subfolders ``middle_finger`` and ``flip-off-google``.

In the ``middle_finger``, you put your middle fingers.

In ``flip-off-google`` you place a ``class_1`` empty subfolder, and a ``none`` subfolder.

In the ``none`` folder you upload the content of [this folder](https://drive.google.com/drive/folders/1-aLmL1egO5l6UPBsY64T7_rigFKOlyG6?usp=drive_link) 📁.

It should look like this:

<img width="200px" src="https://i.ibb.co/C6zfPqR/image.png">

⚠️ ⚠️ ⚠️

In the next cell, you allow access to your Google Drive. Make sure the addresses of the folders' paths are correctly matching your folders' names.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


If you read ``Mounted at /content/drive`` after running the cell, you can move on 🎉

## Step 1 (one). Crop images

In order to train a model on a dataset of middle finger images, we need to clean up each image by cropping the finger, and the finger only.

<img width="50%"  src="https://i.ibb.co/YWWGLbW/Untitled.png">


To do this, we need to install a few extra tools.

[Mediapipe](https://ai.google.dev/edge/mediapipe/solutions/guide) is a machine learning solution by Google. It will help our code *recognise* the hands in each image, and crop them accordingly.

install Mediapipe using the shell-script ``!pip install``

In [ ]:
!pip install mediapipe

We also use ``cv2`` for image processing, ``os`` for file handling, and ``numpy`` for complex numerical operations. These ones are normally already installed in the Google Colab environment you are using.

``import`` all the libraries in python.

> Add blockquote



In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np

Prepare the ``mediapipe`` solutions for your code by creating shortnames for them, like ``mp_hands``, ``hands``, and ``mp_drawing``.

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2)

We want to point our code to the correct Google Drive folders. Our initial dataset, and the folder where we save our cropped images

In [ ]:
input_folder = "/content/drive/MyDrive/flip-off-google/finger"
output_folder = "/content/drive/MyDrive/flip-off-google/gestures_dataset/class_1"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

Since we are preparing the images for training, we also want to make them lighter. That way, the model will <i>digest</i> them faster and more efficiently.

<img width="50%" src="https://i.ibb.co/6RnJwmK/Untitled.png">

Below we can determine a target size.

In [ ]:
target_size = (224, 224)

Now we ``def``ine our function ``add_margin_dynamic()``.

It is a function that, before cropping, adds a margin. So that our hand is visible in full!

In [ ]:
def add_margin_dynamic(x_min, y_min, x_max, y_max, img_shape, margin):
    h, w = img_shape[:2]
    x_margin = int((x_max - x_min) * margin)
    y_margin = int((y_max - y_min) * margin)
    x_min = max(0, x_min - x_margin)
    x_max = min(w, x_max + x_margin)
    y_min = max(0, y_min - y_margin)
    y_max = min(h, y_max + y_margin)
    return x_min, y_min, x_max, y_max

Finally, we *loop* through our ➰ image folder, ➰ read each image, ➰ take the hand position, and ➰ crop + resize.




In [ ]:
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    if img is None:
        print(f"Skipping non-image file: {img_name}")
        continue

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            h, w, _ = img.shape
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * w)
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * w)
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * h)
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * h)

            margin = 0.3
            x_min, y_min, x_max, y_max = add_margin_dynamic(x_min, y_min, x_max, y_max, img.shape, margin)

            cropped_img = img[y_min:y_max, x_min:x_max]

            if cropped_img.size > 0:
                cropped_img_resized = cv2.resize(cropped_img, target_size)
                output_name = os.path.splitext(img_name)[0] + ".jpg"
                output_path = os.path.join(output_folder, output_name)
                cv2.imwrite(output_path, cropped_img_resized)
                print(f"Processed and saved: {output_name}")
    else:
        print(f"No hand detected in: {img_name}")

print("Processing complete!")

Processed and saved: 11.jpg
Processed and saved: 39.jpg
Processed and saved: 35.jpg
Processed and saved: 34.jpg
Processed and saved: 32.jpg
Processed and saved: 33.jpg
Processed and saved: 30.jpg
Processed and saved: 31.jpg
Processed and saved: 27.jpg
Processed and saved: 28.jpg
Processed and saved: 29.jpg
Processed and saved: 3.jpg
Processed and saved: 26.jpg
Processed and saved: 24.jpg
Processed and saved: 18.jpg
Processed and saved: 21.jpg
Processed and saved: 22.jpg
Processed and saved: 2.jpg
Processed and saved: 19.jpg
Processed and saved: 25.jpg
Processed and saved: 23.jpg
Processed and saved: 20.jpg
Processed and saved: 17.jpg
Processed and saved: 16.jpg
Processed and saved: 1.jpg
Processed and saved: 10.jpg
Processed and saved: 0.jpg
Processed and saved: 12.jpg
Processed and saved: 15.jpg
Processed and saved: 13.jpg
Processed and saved: 14.jpg
Processed and saved: 70.jpg
Processed and saved: 69.jpg
Processed and saved: 7.jpg
Processed and saved: 67.jpg
Processed and saved: 68.j

## Step 2 (two). Train a Gesture Recognition model.

In [ ]:
!pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

In [ ]:
dataset_path = "/content/drive/MyDrive/flip-off-google/gestures_dataset"

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

/content/drive/MyDrive/flip-off-google/gestures_dataset
['none', 'class_1']


In [ ]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder


In [ ]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_2 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_2 (ReLU)              (None, 128)               0         
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 2)                 258       
 out (Dense)                                                     
                                                           

In [ ]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

21/21 [==============================] - 1s 3ms/step - loss: 0.1218 - categorical_accuracy: 0.8571
Test loss:0.12184027582406998, Test accuracy:0.8571428656578064


In [ ]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [ ]:
from google.colab import files

files.download('exported_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>